## Carregamento dos Dados

In [9]:
import tensorflow as tf

x_train = tf.keras.utils.image_dataset_from_directory(
    "dataset/train",
    labels="inferred", # Os rótulos são definidos a partir dos nomes das subpastas
    label_mode="categorical", # Formato categórico dos rótulos
    batch_size=32, # Lotes de 32 imagens
    image_size=(128, 128), # Redimensiona as imagens para 128x128
    shuffle=True, # Embaralha as imagens para evitar que a rede aprenda padrões de ordenação
)

x_test = tf.keras.utils.image_dataset_from_directory(
    "dataset/test",
    labels="inferred", # Os rótulos são definidos a partir dos nomes das subpastas
    label_mode="categorical", # Formato categórico dos rótulos
    batch_size=32, # Lotes de 32 imagens
    image_size=(128, 128), # Redimensiona as imagens para 128x128
    shuffle=False, # Não precisa embaralhar nos dados de teste
)

x_val = tf.keras.utils.image_dataset_from_directory(
    "dataset/validation",
    labels="inferred", # Os rótulos são definidos a partir dos nomes das subpastas
    label_mode="categorical", # Formato categórico dos rótulos
    batch_size=32, # Lotes de 32 imagens
    image_size=(128, 128), # Redimensiona as imagens para 128x128
    shuffle=False, # Não precisa embaralhar nos dados de validação
)

Found 7000 files belonging to 7 classes.
Found 1400 files belonging to 7 classes.
Found 1400 files belonging to 7 classes.


## Normalização

In [10]:
# Camada que normaliza as imagens do intervalo [0, 255] para [0, 1]
normalization_layer = tf.keras.layers.Rescaling(1./255)

# Aplicação a normalização às imagens de cada dataset
x_train = x_train.map(lambda x, y: (normalization_layer(x), y))
x_val = x_val.map(lambda x, y: (normalization_layer(x), y))
x_test = x_test.map(lambda x, y: (normalization_layer(x), y))

## Criação do Modelo CNN

Para a criação da CNN foi utilizado o modelo **Sequential**, o qual estabelece uma sequência de camadas que são empilhadas e executadas sequencialmente.

### Camada Convolucional

Essa camada detecta os padrões presentes na imagem e retorna as suas características.

Uma camada `Conv2D` usa filtros, uma pequena matriz de números, que desliza sobre a imagem. Em cada posição é realizado uma operação matemática que retorna um número que representa o quão presente é o padrão daquele filtro na imagem; quanto maior o número mais presente será esse padrão. No final dessa operação é retornado um mapa de características. Se tiver 32 filtros, haverá 32 mapas de características diferentes, cada um destacando um tipo específico de padrão.

Em cada mapa é aplicado a função de ativação `ReLU (max(0, z))`, que transforma todas os negativos em 0.

Na primeira camada Convolucional é aplicado **32 filtros** de matriz **3x3** à imagem original de **128x128**, resultando em **32 mapas de características**. Esses filtros aprenderão caracaterísticas de baixo nível, como bordas e cantos.

Os 32 mapas da camada anterior servirão como entrada para a segunda camada, onde **64 novos filtros** percorrerão esses mapas. Os filtros dessa camada irão combinar as características de baixo nível que foram aprendidas na camada anterior para detectar padrões mais complexos. Ao final dessa camada terá **64 novos mapas de características**.

A terceira camada Convolucional executará o mesmo processo anterior com **128 filtros**, resultando em **128 mapas de características** com padrões ainda mais complexos e abstratos.

### Camada de Pooling

Após cada camada Convolucional é executado uma camada de agrupamento, onde o tamanho dos mapas de características é reduzido. Neste caso, o `MaxPooling2D` divide os mapas em regiões **2x2** e seleciona o valor máximo dentro de cada região.

Ao realizar esse processo, o tamanho dos mapas é reduzido pela metade, mantendo apenas as características mais relevantes. Além disso, a diminuição de parâmetros ajuda a prevenir o overfitting.

### Camada de Flatten

Nessa camada a estrutura 3D das informações que foram passadas pelas camadas anteriores são transformadas em um único vetor 1D. Isso é necessário porque as próximas camadas esperam uma entrada em formato de vetor.

### Camada de Dropout

A técnica de `Dropout` utilizada nessa camada está "desligando" (define como 0) aleatoriamente 50% dos neurônios da camada anterior em cada iteração do treinamento. Isso é necessário porque força o modelo a aprender características mais robustas das imagens e não depender apenas de um único neurônio para tomar a decisão. Essa camada evita que o modelo aprenda demais de quaisquer características ajudando a generalizar melhor, evitando o overfitting.

### Camada Densa

As camadas densas são onde as características aprendidas são combinadas para a decisão final, em que os 128 neurônios realizam um cálculo ponderado de todos os valores de entrada do vetor da camada anterior.

$$Output_j = \sum_{i} (Input_i \times Weight_{ij}) + Bias_j$$

O resultado desse cálculo passa pela função de ativação **ReLU**. Se o valor for positivo, ele permanece o mesmo; se for negativo, torna-se zero.

Por fim, é executado a última camada do modelo, em que os 7 neurônios dessa camada, que correspondem às 7 classes das células, recebem a saída dos 128 neurônios da camada anterior e realizam novamente o cálculo ponderado.

As saídas desses 7 neurônios são passados pela função **Softmax**, que converte os valores em probabilidades, onde cada valor está entre **0 e 1**. A soma das probabilidades de todas as 7 saídas sempre será igual a 1. O neurônio com maior probabilidade indica classe que foi prevista pelo modelo.

In [11]:
# Modelo sequencial onde as camadas são empilhadas uma acima da outra
model = tf.keras.Sequential([
    
    # Primeira camada Convolucional 
    tf.keras.layers.Conv2D(32, # Número de filtros
                           (3, 3), # Tamanho do filtro (3 por 3 pixels)
                           activation='relu', # Função de ativação ReLU
                           input_shape=(128, 128, 3)), # Dimenções das imagens

    # Reduz a resolução da imagem pela metade através do agrupamento guloso de um grupo de 2 por 2 pixels
    tf.keras.layers.MaxPooling2D((2, 2)), 

    # Segunda camada Convolucional 
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    # Terceira camada Convolucional 
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    # Transforma a saída 3D da ultima camada em um vetor de uma dimenção
    tf.keras.layers.Flatten(),
    
    # Evita overfitting desligando 50% dos neurónios de maneira aleatória durante os treinos
    tf.keras.layers.Dropout(0.5),

    # Camada com 128 neurônios conectados
    tf.keras.layers.Dense(128, activation='relu'),
    
    # Camada com o numero de tipos de celulas e função de ativação Softmax (transforma valores em probabilidades)
    tf.keras.layers.Dense(7, activation='softmax')
])

## Compilação do Modelo

In [12]:
# Compila o modelo deixado pronto para treino
model.compile(
    # Otimizador 
    optimizer='adam',
    # Função de perda
    loss='categorical_crossentropy',
    # Métrica para acompanhar a acurácia. 
    metrics=['accuracy']
)
